# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [1]:
# Cleaning the data

import pandas as pd
marketing_customer_analysis_df=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

In [2]:
marketing_customer_analysis_df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [3]:
marketing_customer_analysis_df = marketing_customer_analysis_df.drop(columns=['Unnamed: 0'])

In [4]:
marketing_customer_analysis_df.columns=marketing_customer_analysis_df.columns.str.lower().str.replace(" ", "_")
marketing_customer_analysis_df.rename(columns={"employmentstatus": "employment_status"}, inplace=True)
marketing_customer_analysis_df.dtypes

customer                          object
state                             object
customer_lifetime_value          float64
response                          object
coverage                          object
education                         object
effective_to_date                 object
employment_status                 object
gender                            object
income                             int64
location_code                     object
marital_status                    object
monthly_premium_auto               int64
months_since_last_claim          float64
months_since_policy_inception      int64
number_of_open_complaints        float64
number_of_policies                 int64
policy_type                       object
policy                            object
renew_offer_type                  object
sales_channel                     object
total_claim_amount               float64
vehicle_class                     object
vehicle_size                      object
vehicle_type    

In [5]:
marketing_customer_analysis_df['effective_to_date']=pd.to_datetime(marketing_customer_analysis_df['effective_to_date'],format='%m/%d/%y' )
marketing_customer_analysis_df.head(1)

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,NaN


In [6]:
marketing_customer_analysis_df.shape

(10910, 25)

In [7]:
marketing_customer_analysis_df.isnull().sum()

customer                            0
state                             631
customer_lifetime_value             0
response                          631
coverage                            0
education                           0
effective_to_date                   0
employment_status                   0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto                0
months_since_last_claim           633
months_since_policy_inception       0
number_of_open_complaints         633
number_of_policies                  0
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                  0
vehicle_class                     622
vehicle_size                      622
vehicle_type                     5482
dtype: int64

In [8]:
marketing_customer_analysis_df=marketing_customer_analysis_df.drop(columns= "vehicle_type")

In [9]:
marketing_customer_analysis_df_afterdropna = marketing_customer_analysis_df.dropna()

In [10]:
marketing_customer_analysis_df_afterdropna.shape

(9134, 24)

In [11]:
marketing_customer_analysis_df_afterdropna.isnull().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employment_status                0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [12]:
marketing_customer_analysis_df_afterdropna.duplicated().sum()

0

In [13]:
mca_cleaned=marketing_customer_analysis_df_afterdropna

In [14]:
# 1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 
# and have a response of "Yes" to the last marketing campaign.

new_data_frame=mca_cleaned[(mca_cleaned["total_claim_amount"] > 1000) & (mca_cleaned["response"] == "Yes")]
new_data_frame.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
189,OK31456,California,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,43,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,43,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize
419,GW43195,Oregon,25807.063000,Yes,Extended,College,2011-02-13,Employed,F,71210,...,89,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,16181,...,79,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,...,5,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize


In [15]:
new_data_frame.tail()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
10237,FD73388,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,16181,...,79,0.0,8,Personal Auto,Personal L3,Offer1,Web,1261.319869,SUV,Medsize
10351,FN44127,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,20978,...,96,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small
10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2011-02-08,Employed,M,55687,...,91,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize
10487,IX60941,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,20978,...,96,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small
10708,CK39096,Oregon,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,...,5,0.0,1,Personal Auto,Personal L3,Offer2,Web,1027.000029,SUV,Medsize


In [16]:
# 2. Using the original Dataframe, analyze the average total_claim_amount by each policy type
# and gender for customers who have responded "Yes" to the last marketing campaign
# Write your conclusions.

customers_yes_df= mca_cleaned[mca_cleaned['response']=="Yes"]
avg_TCA_by_policy_gender=customers_yes_df.groupby(["policy_type","gender"])["total_claim_amount"].mean()
print(avg_TCA_by_policy_gender)

policy_type     gender
Corporate Auto  F         431.480117
                M         412.756888
Personal Auto   F         454.089997
                M         453.603715
Special Auto    F         455.648867
                M         414.799291
Name: total_claim_amount, dtype: float64


In [17]:
type(avg_TCA_by_policy_gender)

pandas.core.series.Series

In [18]:
avg_TCA_by_policy_gender=avg_TCA_by_policy_gender.reset_index()
print("Column names before renaming:", avg_TCA_by_policy_gender.columns)

Column names before renaming: Index(['policy_type', 'gender', 'total_claim_amount'], dtype='object')


In [19]:
avg_TCA_by_policy_gender.columns = ['policy_type', 'gender','average_total_claim_amount']
avg_TCA_by_policy_gender

,policy_type,gender,average_total_claim_amount
0,Corporate Auto,F,431.480117
1,Corporate Auto,M,412.756888
2,Personal Auto,F,454.089997
3,Personal Auto,M,453.603715
4,Special Auto,F,455.648867
5,Special Auto,M,414.799291


On average total claim amount varies from 412 to 456. In each policy_type Females pay a little bit more on average then Males with highest difference in policy_type "Special Auto". For some deeper analysis we need to dive deeper into the reasons of these trends.

In [20]:
# 3. Analyze the total number of customers who have policies in each state, 
# and then filter the results to only include states where there are more than 500 customers.

customer_per_states= mca_cleaned.groupby('state').size()
print(customer_per_states)

state
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
dtype: int64


In [21]:
states_over_500= customer_per_states[customer_per_states > 500]
states_over_500

state
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
dtype: int64

In [22]:
# 4. Find the maximum, minimum, and median customer lifetime value by education level and gender.
# Write your conclusions.
CLV_max_min_median=mca_cleaned.groupby(["education","gender"])["customer_lifetime_value"].agg(['max','min','median']).reset_index()
CLV_max_min_median

,education,gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5678.050167
1,Bachelor,M,67907.27050,1898.007675,5555.830205
2,College,F,61850.18803,1898.683686,5621.789489
3,College,M,61134.68307,1918.119700,5989.773931
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5620.587184
6,High School or Below,F,55277.44589,2144.921535,6044.020038
7,High School or Below,M,83325.38119,1940.981221,6176.700392
8,Master,F,51016.06704,2417.777032,5801.128252
9,Master,M,50568.25912,2272.307310,5617.956917


The customer lifetime value (CLV) varies widely, from as low as 1898 to as high as 83,326. However, the median CLV values, typically between 5332 and 6,177, suggest most customers are closer to the lower end. This means there are a few customers with very high CLVs, but many are on the lower side.
Education trend shows that while higher education like Bachelor's tend to have higher maximum CLVs value, the median suggest that most customer across all education levels have relatively simmilar lifetime values.
Gender trend shows that CLV in median values across education levels are quite similar between genders.

In [23]:
mca_cleaned.head(1)

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,48029,...,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize


In [24]:
#5. The marketing team wants to analyze the number of policies sold by state and month. 
# Present the data in a table where the months are arranged as columns and the states are arranged as rows.
# adding a new "month" column
mca_cleaned.loc[:,'month']=mca_cleaned["effective_to_date"].dt.month
mca_cleaned.head(1)

C:\Users\Tijana&Ivan\AppData\Local\Temp\ipykernel_13332\562984512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mca_cleaned.loc[:,'month']=mca_cleaned["effective_to_date"].dt.month


,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,month
0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,2


In [25]:
# creating a pivot table
policy_sold_by_state_month=mca_cleaned.pivot_table(index='state', 
    columns='month', 
    values='policy', 
    aggfunc='count',
    fill_value=0
).rename_axis(columns='Month')
print(policy_sold_by_state_month)


Month          1     2
state                 
Arizona      899   804
California  1695  1455
Nevada       494   388
Oregon      1396  1205
Washington   414   384


In [26]:
# 6.Display a new DataFrame that contains the number of policies sold by month, by state, 
#   for the top 3 states with the highest number of policies sold.

# *Hint:*
#- *To accomplish this, you will first need to group the data by state and month, 
    #then count the number of policies sold for each group. Afterwards, 
    #you will need to sort the data by the count of policies sold in descending order.*
#- *Next, you will select the top 3 states with the highest number of policies sold.*
#- *Finally, you will create a new DataFrame that contains the number of policies sold by month
    #for each of the top 3 states.*


# Counting the number of policies per state and month
policy_number= mca_cleaned.groupby(["state","month"]).size().reset_index(name="policy_count")
policy_number


,state,month,policy_count
0,Arizona,1,899
1,Arizona,2,804
2,California,1,1695
3,California,2,1455
4,Nevada,1,494
5,Nevada,2,388
6,Oregon,1,1396
7,Oregon,2,1205
8,Washington,1,414
9,Washington,2,384


In [27]:
# Identify the top 3 states by total number of policies sold
total_policies_by_state= policy_number.groupby('state')['policy_count'].sum().sort_values(ascending= False)
top_three_states= total_policies_by_state.nlargest(3).index
top_three_states

Index(['California', 'Oregon', 'Arizona'], dtype='object', name='state')

In [28]:
# Filtering the original grouped data to include only the top 3 states
top_three_states_data = policy_number[policy_number['state'].isin(top_three_states)]
top_three_states_data

,state,month,policy_count
0,Arizona,1,899
1,Arizona,2,804
2,California,1,1695
3,California,2,1455
6,Oregon,1,1396
7,Oregon,2,1205


In [29]:
# Pivot to create a DataFrame where months are columns and states are rows
top_three_states_data_pivot = top_three_states_data.pivot_table(
    index='state',
    columns='month',
    values='policy_count',
    fill_value=0,
    aggfunc='sum'
).rename_axis(columns='Month')

print(top_three_states_data_pivot)

Month          1     2
state                 
Arizona      899   804
California  1695  1455
Oregon      1396  1205


In [30]:
# 7.The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
    #Hint: You can use melt to unpivot the data and create a table that shows the customer response rate 
    #(those who responded "Yes") by marketing channel.

# Step 1: Filter to find the total number of customers per channel
total_responses_per_channel = marketing_customer_analysis_df.groupby('sales_channel').size()
total_responses_per_channel

sales_channel
Agent          4121
Branch         3022
Call Center    2141
Web            1626
dtype: int64

In [33]:
# Step 2: Count "Yes" responses per channel
yes_responses_per_channel = marketing_customer_analysis_df[
    marketing_customer_analysis_df['response'].str.lower() == 'yes'
].groupby('sales_channel').size()
yes_responses_per_channel

sales_channel
Agent          742
Branch         326
Call Center    221
Web            177
dtype: int64

In [34]:
# Step 3: Calculate response rate
response_rate_per_channel = (yes_responses_per_channel / total_responses_per_channel) * 100
response_rate_per_channel

sales_channel
Agent          18.005339
Branch         10.787558
Call Center    10.322279
Web            10.885609
dtype: float64

In [35]:
# Step 4: Create a DataFrame to display results nicely
response_rate_df = pd.DataFrame({
    'Total Responses': total_responses_per_channel,
    'Yes Responses': yes_responses_per_channel,
    'Response Rate (%)': response_rate_per_channel
}).fillna(0)  # Fill NaN values in case any channel had no responses

# Display the table
print(response_rate_df)

               Total Responses  Yes Responses  Response Rate (%)
sales_channel                                                   
Agent                     4121            742          18.005339
Branch                    3022            326          10.787558
Call Center               2141            221          10.322279
Web                       1626            177          10.885609
